In [108]:
#https://moma.corp.google.com/person/joeysilva #tl lead for sic
#set local vars
PROJECT_ID = "#change me" #change me
REGION="us-central1" 
DATASETID="#change me" #change me
VIEWNAME="VW_CLOUD_BILLING" 
BUCKETNAME="#change me" #change me

In [109]:
#generate random name for cloud run job
import random
import string
DISPLAY_NAME = 'cloud-billing-forecasting-'
JOB_NAME = DISPLAY_NAME + "".join(random.choices(string.ascii_lowercase + string.digits, k=5))

In [110]:
JOB_NAME

'cloud-billing-forecasting-60gkq'

In [111]:
%%capture
#build container image
#note "." at end of glcoud command which tells builds to use source code from same/local directory.  e.g. main.py
!gcloud builds submit --pack image=gcr.io/$PROJECT_ID/$JOB_NAME .

In [112]:
#create the cloud run job 
#note use of set-env-vars which sets os.vars on container image
!gcloud beta run jobs create --quiet  $JOB_NAME \
    --image gcr.io/$PROJECT_ID/$JOB_NAME\
    --region us-central1 \
    --command web \
    --task-timeout=60m \
    --set-env-vars projectid_=PROJECT_ID  \
    --set-env-vars region_=REGION \
    --set-env-vars datasetid_=DATASETID \
    --set-env-vars viewname_=VIEWNAME \
    --set-env-vars bucketname_=BUCKETNAME

Creating Cloud Run job [cloud-billing-forecasting-60gkq] in project [g-sql-morphic-luminous] region [us-central1]
Creating job...                                                                
  Creating job...                                                              
⠛ Creating job...                                                              
⠹ Creating job...                                                              
⠼ Creating job...                                                              
⠶ Creating job...                                                              
⠧ Creating job...                                                              
⠏ Creating job...                                                              
⠛ Creating job...                                                              
⠹ Creating job...                                                              
⠼ Creating job...                                                              
⠶ Crea

In [113]:
%%capture
#run the job
!gcloud beta run jobs execute --wait --quiet --region='us-central1' $JOB_NAME

In [114]:
#display response from job run
response = !gcloud beta run jobs describe $JOB_NAME --region us-central1 --format="value(status.latestCreatedExecution.name)"
str = response.get_spstr()
!gcloud beta run jobs executions --region='us-central1' describe $str

X Execution cloud-billing-forecasting-60gkq-chjgh in region us-central1
0 tasks completed successfully
1 task failed to complete
Elapsed time: 1 minute and 31 seconds
 
Log URI: https://console.cloud.google.com/logs/viewer?project=g-sql-morphic-luminous&advancedFilter=resource.type%3D%22cloud_run_job%22%0Aresource.labels.job_name%3D%22cloud-billing-forecasting-60gkq%22%0Aresource.labels.location%3D%22us-central1%22%0Alabels.%22run.googleapis.com/execution_name%22%3D%22cloud-billing-forecasting-60gkq-chjgh%22
 
Image:           gcr.io/g-sql-morphic-luminous/cloud-billing-forecasting-60gkq at db021c1c...
Tasks:           1
Command:         web
Memory:          512Mi
CPU:             1000m
Task Timeout:    3600s
Max Retries:     3
Parallelism:     1
Service account: 732814971409-compute@developer.gserviceaccount.com
Env vars:
  bucketname_    BUCKETNAME
  datasetid_     DATASETID
  projectid_     PROJECT_ID
  region_        REGION
  viewname_      VIEWNAME
X Task cloud-billing-forecasting